In [1]:
import numpy as np
import cv2
from fractions import Fraction
from sympy import diff, Symbol
import sympy
import numpy as np
import math
from sympy import *
def Distance(x1,y1,z1,x2,y2,z2):
    dist = sympy.sqrt((x2 - x1)**2 + (y2 - y1)**2+(z2 - z1)**2)
    return dist

        
# construct the argument parse and parse the arguments
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5, 5))
# load the image
image = cv2.imread('Long.png')
#image = cv2.resize(image, (1000, 750))
#cv2.imshow("images", image)
# define the list of boundaries
thres = 40
boundaries = [
	([0, 0, 230], [50, 50, 255]),#red
    ([0, 100, 230], [30, 180, 255]),#orange
    ([230, 0, 0], [255, 50,50 ]) # blue    
]
clone = image.copy()
# loop over the boundaries
ax=[]
ay=[]
for (lower, upper) in boundaries:
    lower = np.array(lower, dtype = "uint8")
    upper = np.array(upper, dtype = "uint8")
	# find the colors within the specified boundaries and apply
	# the mas
    mask = cv2.inRange(image, lower, upper)
    output1 = cv2.bitwise_and(image, image, mask = mask)
    x = output1.nonzero()
    ax.append(x[1][0])
    ay.append(x[0][0])
x_center=[]
y_center=[]
for i in ax:
    x_center.append(i-2000)
for i in ay:
    y_center.append(i-1500)
print("x_center",x_center)
print("y_center",y_center)
f=4.73#mm
sensor_size=[6.4,4.8]#mm
X1,Y1,Z1=0,0,0#A
X2,Y2,Z2=27.5,5.8,0#B cm
ratio=Distance(X1,Y1,Z1,X2,Y2,Z2)/(Distance(x_center[0],y_center[0],0,x_center[1],y_center[1],0)*1.6*0.0001)#cm
print("ratio",int(ratio))
S2real=f*0.1*ratio#cm
for i in range(5):
    S = symbols('S')
    S=solve((1/S)+(1/S2real)-(1/f),S)[0]
    #print("S",S)
    S2real = symbols('S2real')
    S2real=solve((1/S)+(1/S2real)-(1/f),S2real)[0]
print("S2real",S2real)#cm   
print("S",S)#cm

x_center [-313, 477, 270]
y_center [342, 211, 607]
ratio 219
S2real 103.754526306742
S 4.95593291616157


In [2]:
def Distance_o(x1,y1,S2real):#for calculate the distance of the pixel
    dist = sympy.sqrt((sympy.sqrt((x1**2+y1**2))*ratio*0.1)**2+S**2)
    return dist
X3,Y3,Z3=21,-8.3,0#cm
camera_real_x,camera_real_y,camera_real_z=(X1+X2+X3)/3,(Y1+Y2+Y3)/3,S2real#initial pose for camera cm

x1,y1=x_center[0],y_center[0]#pixel
x2,y2=x_center[1],y_center[1]#pixel
x3,y3=x_center[2],y_center[2]#pixel

A = Symbol('A',real=True)
B = Symbol('B',real=True)
C = Symbol('C',real=True)
angle1=sympy.acos(np.dot((X1-A,Y1-B,Z1-C),(X2-A,Y2-B,Z2-C))/(Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X2,Y2,Z2)))
angle2=sympy.acos(np.dot((x1*ratio*0.1-A,y1*ratio*0.1-B,(S+S2real)-C),(x2*ratio*0.1-A,y2*ratio*0.1-B,(S+S2real)-C))/(Distance_o(x1,y1,S)*Distance_o(x2,y2,S)))
angle3=sympy.acos(np.dot((X2-A,Y2-B,Z2-C),(X3-A,Y3-B,Z3-C))/(Distance(A,B,C,X2,Y2,Z2)*Distance(A,B,C,X3,Y3,Z3)))
angle4=sympy.acos(np.dot((x2*ratio*0.1-A,y2*ratio*0.1-B,(S+S2real)-C),(x3*ratio*0.1-A,y3*ratio*0.1-B,(S+S2real)-C))/(Distance_o(x2,y2,S)*Distance_o(x3,y3,S)))

delt_A=diff(angle1-angle2, A)
delt_B=diff(angle1-angle2, B)
delt_C=diff(angle1-angle2, C)

delt_A_2=diff(angle3-angle4, A)
delt_B_2=diff(angle3-angle4, B)
delt_C_2=diff(angle3-angle4, C)

for i in range(500):
    delt_A=delt_A.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_B=delt_B.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_C=delt_C.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_A_2=delt_A.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_B_2=delt_B.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    delt_C_2=delt_C.subs({A:camera_real_x,B:camera_real_y,C:camera_real_z})
    camera_real_x=camera_real_x-10*delt_A-10*delt_A_2
    camera_real_y=camera_real_y-10*delt_B-10*delt_B_2
    camera_real_z=camera_real_z-1*delt_C-1*delt_C_2
    #print(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2))<1)
    angle1=sympy.acos(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2)))
    angle2=sympy.acos(np.dot((x1*ratio*0.1-camera_real_x,y1*ratio*0.1-camera_real_y,(S+S2real)-camera_real_z),(x2*ratio*0.1-camera_real_x,y2*ratio*0.1-camera_real_y,(S+S2real)-camera_real_z))/(Distance_o(x1,y1,S)*Distance_o(x2,y2,S)))
    #angle3=sympy.acos(np.dot((X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z),(X3-camera_real_x,Y3-camera_real_y,Z3-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2)*Distance(camera_real_x,camera_real_y,camera_real_z,X3,Y3,Z3)))

    print("x=%f y=%f z=%f"%(camera_real_x,camera_real_y,camera_real_z))
    
    if Abs(angle1-angle2)<0.01:
        print("converge x=%f y=%f z=%f"%(camera_real_x,camera_real_y,camera_real_z))
        break
    
        
    
    

x=16.169245 y=-0.832816 z=103.759638
x=16.171824 y=-0.832298 z=103.764750
x=16.174402 y=-0.831780 z=103.769862
x=16.176981 y=-0.831263 z=103.774975
x=16.179560 y=-0.830745 z=103.780087
x=16.182138 y=-0.830227 z=103.785199
x=16.184717 y=-0.829710 z=103.790311
x=16.187296 y=-0.829192 z=103.795423
x=16.189874 y=-0.828674 z=103.800535
x=16.192453 y=-0.828157 z=103.805647
x=16.195031 y=-0.827639 z=103.810759
x=16.197610 y=-0.827121 z=103.815871
x=16.200189 y=-0.826604 z=103.820983
x=16.202767 y=-0.826086 z=103.826095
x=16.205346 y=-0.825568 z=103.831207
x=16.207924 y=-0.825051 z=103.836319
x=16.210503 y=-0.824533 z=103.841431
x=16.213082 y=-0.824015 z=103.846543
x=16.215660 y=-0.823498 z=103.851655
x=16.218239 y=-0.822980 z=103.856768
x=16.220817 y=-0.822462 z=103.861880
x=16.223396 y=-0.821945 z=103.866992
x=16.225975 y=-0.821427 z=103.872104
x=16.228553 y=-0.820909 z=103.877216
x=16.231132 y=-0.820392 z=103.882328
x=16.233710 y=-0.819874 z=103.887440
x=16.236289 y=-0.819356 z=103.892552
x

x=16.741696 y=-0.717895 z=104.894516
x=16.744274 y=-0.717377 z=104.899628
x=16.746853 y=-0.716860 z=104.904740
x=16.749432 y=-0.716342 z=104.909852
x=16.752010 y=-0.715824 z=104.914964
x=16.754589 y=-0.715307 z=104.920076
x=16.757167 y=-0.714789 z=104.925188
x=16.759746 y=-0.714271 z=104.930300
x=16.762325 y=-0.713754 z=104.935412
x=16.764903 y=-0.713236 z=104.940524
x=16.767482 y=-0.712718 z=104.945636
x=16.770060 y=-0.712201 z=104.950748
x=16.772639 y=-0.711683 z=104.955860
x=16.775218 y=-0.711165 z=104.960972
x=16.777796 y=-0.710648 z=104.966084
x=16.780375 y=-0.710130 z=104.971197
x=16.782953 y=-0.709612 z=104.976309
x=16.785532 y=-0.709095 z=104.981421
x=16.788111 y=-0.708577 z=104.986533
x=16.790689 y=-0.708059 z=104.991645
x=16.793268 y=-0.707542 z=104.996757
x=16.795846 y=-0.707024 z=105.001869
x=16.798425 y=-0.706506 z=105.006981
x=16.801004 y=-0.705989 z=105.012093
x=16.803582 y=-0.705471 z=105.017205
x=16.806161 y=-0.704953 z=105.022317
x=16.808739 y=-0.704436 z=105.027429
x

x=17.314146 y=-0.602974 z=106.029393
x=17.316725 y=-0.602456 z=106.034505
x=17.319303 y=-0.601939 z=106.039617
x=17.321882 y=-0.601421 z=106.044729
x=17.324461 y=-0.600903 z=106.049841
x=17.327039 y=-0.600386 z=106.054953
x=17.329618 y=-0.599868 z=106.060065
x=17.332196 y=-0.599350 z=106.065177
x=17.334775 y=-0.598833 z=106.070289
x=17.337354 y=-0.598315 z=106.075401
x=17.339932 y=-0.597798 z=106.080514
x=17.342511 y=-0.597280 z=106.085626
x=17.345089 y=-0.596762 z=106.090738
x=17.347668 y=-0.596245 z=106.095850
x=17.350247 y=-0.595727 z=106.100962
x=17.352825 y=-0.595209 z=106.106074
x=17.355404 y=-0.594692 z=106.111186
x=17.357983 y=-0.594174 z=106.116298
x=17.360561 y=-0.593656 z=106.121410
x=17.363140 y=-0.593139 z=106.126522
x=17.365718 y=-0.592621 z=106.131634
x=17.368297 y=-0.592103 z=106.136746
x=17.370876 y=-0.591586 z=106.141858
x=17.373454 y=-0.591068 z=106.146970
x=17.376033 y=-0.590550 z=106.152082
x=17.378611 y=-0.590033 z=106.157194
x=17.381190 y=-0.589515 z=106.162306
x

In [3]:
camera_real_x,camera_real_y,camera_real_z=56.211932,15.395214,3.000000
X1,Y1,Z1=100,100,33
X2,Y2,Z2=200,200,33
x1,y1=10,10
x2,y2=20,20
angle1=sympy.acos(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2)))
angle2=sympy.acos(np.dot((x1-camera_real_x,y1-camera_real_y,0-camera_real_z),(x2-camera_real_x,x2-camera_real_y,0-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,x1,y1,0)*Distance(camera_real_x,camera_real_y,camera_real_z,x2,y2,0)))
print(np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/(Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2))<1)

print(angle1-angle2)

True
0.00984904461032987


In [4]:
def Distance(x1,y1,z1,x2,y2,z2):
    dist = sympy.sqrt((x2 - x1)**2 + (y2 - y1)**2+(z2 - z1)**2)
    return dist
A = Symbol('A')
B = Symbol('B')
C = Symbol('C')

X1,Y1,Z1=100,100,30
X2,Y2,Z2=100,(100+30/sympy.sqrt(3)),30
x1,y1=1,1
x2,y2=2,2


angle1=np.dot((X1-A,Y1-B,Z1-C),(X2-A,Y2-B,Z2-C))/Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X2,Y2,Z2)
angle2=np.dot((x1-A,y1-B,0-C),(x2-A,x2-B,0-C))/Distance(A,B,C,x1,y1,0)*Distance(A,B,C,x2,y2,0)


a=diff(angle1-angle2, A).subs({A:5,B:10,C:3})
print(a+5)
camera_real_x,camera_real_y,camera_real_z=100,100,0

angle1=np.dot((X1-camera_real_x,Y1-camera_real_y,Z1-camera_real_z),(X2-camera_real_x,Y2-camera_real_y,Z2-camera_real_z))/Distance(camera_real_x,camera_real_y,camera_real_z,X1,Y1,Z1)*Distance(camera_real_x,camera_real_y,camera_real_z,X2,Y2,Z2)
angle2=np.dot((x1-camera_real_x,y1-camera_real_y,0-camera_real_z),(x2-camera_real_x,x2-camera_real_y,0-camera_real_z))/Distance(camera_real_x,camera_real_y,camera_real_z,x1,y1,0)*Distance(camera_real_x,camera_real_y,camera_real_z,x2,y2,0)
#print(type(angle1))
print("angle1=%f angle2=%f"%(angle1,angle2))

abs(angle1-angle2)<1
X = Symbol('X',real=True)

diff(sympy.Abs(X**2+2), X).subs({X:5}).n(10)

-95*sqrt(17854)*sqrt(9754 + (10*sqrt(3) + 90)**2)/8927 - 95*sqrt(17854)*(900*sqrt(3) + 17854)/(17854*sqrt(9754 + (10*sqrt(3) + 90)**2)) - 29957*sqrt(2173)/230338 + 5 + 95*sqrt(17854)*sqrt(9754 + (10*sqrt(3) + 90)**2)*(900*sqrt(3) + 17854)/318765316
angle1=1039.230485 angle2=19208.000000


10.00000000

In [5]:
X1,Y1,Z1=0,0,30
X2,Y2,Z2=0,(30/sympy.sqrt(3)),0
A,B,C=0,0,0
angle1=sympy.acos(np.dot((X1-A,Y1-B,Z1-C),(X2-A,Y2-B,Z2-C))/Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X2,Y2,Z2))
print("angle1=%f"% angle1)
angle1<1
print(angle1.n(10))
sympy.acos(0.2)

angle1=1.570796
1.570796327


1.36943840600457

In [6]:
X1,Y1,Z1=0,0,30
X2,Y2,Z2=0,0,0
A,B,C=0,(30/sympy.sqrt(3)),0
angle1=np.dot((X1-A,Y1-B,Z1-C),(X2-A,Y2-B,Z2-C))/(Distance(A,B,C,X1,Y1,Z1)*Distance(A,B,C,X2,Y2,Z2))
print(Distance(A,B,C,X1,Y1,Z1),Distance(A,B,C,X2,Y2,Z2))
print(angle1)
sympy.acos(angle1)

20*sqrt(3) 10*sqrt(3)
1/2


pi/3